In [8]:
from nltk.tokenize import TreebankWordTokenizer
import os
import gensim
from sklearn.model_selection import train_test_split
import time
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv1D, Dropout, GlobalMaxPooling1D
from keras.optimizers import SGD
import gensim
from keras.models import model_from_json
import pickle
import pandas as pd

from model_template import Model
from glove_keras_cnn import glove_keras_cnn

In [2]:
load = True  
maxlen = 100
batch_size = 32
embedding_dims = 300
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 4
num_classes = 3

In [6]:
def pad_trunc(data, maxlen):
    new_data = []
    # Create a vector of 0s the length of our word vectors
    zero_vector = []
    for _ in range(len(data[0][0])):
        zero_vector.append(0.0)
    for sample in data:
        if len(sample) > maxlen:
            temp = sample[:maxlen]
        elif len(sample) < maxlen:
            temp = sample
            # Append the appropriate number 0 vectors to the list
            additional_elems = maxlen - len(sample)
            for _ in range(additional_elems):
                temp.append(zero_vector)
        else:
            temp = sample
        new_data.append(temp)
    return new_data

In [9]:
filename = 'train.csv'
df = pd.read_csv(filename, index_col='id')
authors = list(df.author.unique())
lookup = {a: _ for _, a in enumerate(authors)}
y_numbers = [lookup[i] for i in df.author]
y_vecs = []
for y in y_numbers:
    base_vec = np.zeros(num_classes, dtype='int')
    base_vec[y] = 1
    y_vecs.append(base_vec)
df['y'] = y_vecs

In [64]:
authors[0]

'EAP'

In [19]:
EAP_test_string = """Once upon a midnight dreary, while I pondered, weak and weary, 
Over many a quaint and curious volume of forgotten lore, 
While I nodded, nearly napping, suddenly there came a tapping, 
As of some one gently rapping, rapping at my chamber door.""".replace("\n", "")

In [20]:
HPL_test_string = """In this luminous Company I was tolerated more because of my Years 
than for my Wit or Learning; being no Match at all for the rest. My Friendship for the 
celebrated Monsieur Voltaire was ever a Cause of Annoyance to the Doctor; who was deeply 
orthodox, and who us'd to say of the French Philosopher.""".replace("\n", "")

In [21]:
MWS_test_string = """A few seconds ago they had all been active and healthy beings, 
so full of employment they could not afford to mend his calèche unless tempted by 
some extraordinary reward; now the men declared themselves cripples and invalids, the 
children were orphans, the women helpless widows, and they would all die of hunger if 
his Eccellenza did not bestow a few grani.""".replace("\n", "")

In [ ]:
GLOVE_DIR = "/media/D/data/glove/"
GLOVE_W2V_FILE = "glove.840B.300d.w2vformat.txt"
GLOVE_W2V_PATH = os.path.join(GLOVE_DIR, GLOVE_W2V_FILE)
glove_model = gensim.models.KeyedVectors.load_word2vec_format(GLOVE_W2V_PATH)
wv = glove_model.wv
tokenizer = TreebankWordTokenizer()

In [38]:
vectorized_data = []
for sentence in [EAP_test_string, HPL_test_string, MWS_test_string]:
    sample_vecs = []
    for token in tokenizer.tokenize(sentence):
        try:
            sample_vecs.append(wv[token])
        except KeyError:
            # print(token, "not in wv")
            pass
    vectorized_data.append(sample_vecs)
pickle.dump(vectorized_data, open("glove_vectorized_test_sentences", "wb"))

In [39]:
vectorized_data = pickle.load(open("glove_vectorized_test_sentences", "rb"))
vectorized_data = pad_trunc(vectorized_data, maxlen)

In [40]:
len(vectorized_data[2])

100

In [42]:
test = np.asarray(vectorized_data)
test.shape

(3, 100, 300)

In [43]:
new_test = np.reshape(test, (len(test), maxlen, embedding_dims))

In [44]:
new_test.shape

(3, 100, 300)

In [45]:
cnn = glove_keras_cnn()
model = cnn.load()

In [48]:
predictions = model.predict(new_test)

In [61]:
np.argmax(predictions, axis=1)

array([0, 0, 2])